<a href="https://colab.research.google.com/github/Zoyaaaaaaa/EduMail/blob/main/EMAIL_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An edtech platform email categorizer

In [1]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip show langgraph

Name: langgraph
Version: 0.2.53
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 


In [3]:
import os

# Replace with the actual API keys
os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

# Verify that the keys are set correctly
print("GROQ_API_KEY:", os.environ.get("GROQ_API_KEY"))
print("TAVILY_API_KEY:", os.environ.get("TAVILY_API_KEY"))


GROQ_API_KEY: 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
TAVILY_API_KEY: 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


In [4]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [6]:

def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [7]:
# Categorize Email for EdTech Platform
email_categorizer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an Email Categorizer Agent for an edtech platform. Your task is to analyze customer emails and categorize them based on their content.

    Categorize the email into one of the following categories:
        course_inquiry - used when someone is asking about courses, syllabus, or schedules.
        platform_issue - used when someone is reporting a technical issue with the platform.
        feedback - used when someone provides feedback about the platform, courses, or instructors.
        payment_issue - used when someone is reporting problems with payments, refunds, or billing.
        off_topic - used when the email does not relate to any of the categories above.

    Respond with only the category name from the options above.

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = email_categorizer_prompt | GROQ_LLM | StrOutputParser()

# Example Email for EdTech
EMAIL = """Hi there, \n
I am facing issues with accessing my enrolled course on your platform. It keeps showing an error message. \n

Can you help? \n
Thanks,
John
"""

# Generate the Email Category
result = email_category_generator.invoke({"initial_email": EMAIL})
print(f"Email Category: {result}")


Email Category: platform_issue


In [8]:
# Research Router for EdTech Platform
research_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading customer emails for an edtech platform and deciding whether to route it for research or draft an email directly.

    Use the following criteria to decide:
    - If the email only requires a simple response or contains basic feedback (e.g., "thank you"), choose 'draft_email'.
    - If the email contains technical questions, complex feedback, or issues needing detailed research, choose 'research_info'.

    Return a JSON with a single key 'router_decision' and no preamble or explanation.

    Email to route: INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id>""",
    input_variables=["initial_email", "email_category"],
)

research_router = research_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = "platform_issue"  # Example from Categorization
research_router_result = research_router.invoke({"initial_email": EMAIL, "email_category": email_category})
print(f"Research Router Decision: {research_router_result}")


Research Router Decision: {'router_decision': 'research_info'}


In [9]:
# Search Keywords for EdTech Platform
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at identifying search keywords for an edtech platform.

    Given the INITIAL_EMAIL and EMAIL_CATEGORY, determine the best 3 keywords to help draft a response or find relevant information.

    Return a JSON with a single key 'keywords' containing no more than 3 keywords.

    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id>""",
    input_variables=["initial_email", "email_category"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | JsonOutputParser()

search_keyword_result = search_keyword_chain.invoke({"initial_email": EMAIL, "email_category": email_category})
print(f"Search Keywords: {search_keyword_result}")


Search Keywords: {'keywords': ['course access', 'error message', 'platform issue']}


In [11]:
## Write Draft Email
draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent for an EdTech platform. Write a helpful email in a thoughtful and friendly way based on the INITIAL_EMAIL, EMAIL_CATEGORY, and RESEARCH_INFO.

    Follow these guidelines:
    - For 'course_enquiry': Provide information about the course syllabus, prerequisites, and benefits.
    - For 'technical_issue': Acknowledge the issue and provide clear steps to resolve it.
    - For 'feedback': Acknowledge and thank the user for their feedback.
    - For 'pricing_query': Provide detailed pricing or fee structures.
    - For 'general_query': Respond appropriately based on the email content.

    Always end the email with a polite sign-off and your role.

    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email", "email_category", "research_info"],
)


draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'platform_issue'
research_info = None

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category,"research_info":research_info}))

OutputParserException: Invalid json output: Here is a helpful email in response to John's query:

Hi John,

I'm so sorry to hear that you're facing issues accessing your enrolled course on our platform. I can imagine how frustrating that must be for you.

I'd be happy to help you resolve this issue as quickly as possible. To assist you better, could you please provide me with more details about the error message you're seeing? Is it a specific error code or message? Additionally, have you tried clearing your browser cache, updating your browser, or checking if the issue persists on a different device?

In the meantime, I've notified our technical team to investigate this issue further. We'll do our best to resolve it within the next 24-48 hours.

If you need any immediate assistance or have any further questions, please don't hesitate to reach out to me directly. Your learning experience is our top priority, and I'm committed to getting you back on track.

Thank you for your patience a

In [13]:
rewrite_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Decide if a draft email for an EdTech platform needs rewriting. Use the following criteria:
    - Does it address the query clearly and completely?
    - Is it professional and aligned with EdTech standards?

    Return 'rewrite' or 'no_rewrite' in a JSON with a single key 'router_decision'.

    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    DRAFT_EMAIL: {draft_email} \n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email", "email_category", "draft_email"],
)
rewrite_router = rewrite_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'platform_issue'
draft_email = "Yo we can't help you, best regards Sarah"

print(rewrite_router.invoke({"initial_email": EMAIL, "email_category": email_category, "draft_email": draft_email}))


{'router_decision': 'rewrite'}


In [15]:
## Draft Email Analysis
draft_analysis_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Analyze the draft email for an EdTech platform query. Provide constructive feedback:
    - Does the draft address the query effectively?
    - Suggest improvements to make the email more professional or informative.

    Return a JSON with a single key 'draft_analysis'.

    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    DRAFT_EMAIL: {draft_email} \n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email", "email_category", "research_info", "draft_email"],
)



draft_analysis_chain = draft_analysis_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"
email_analysis = draft_analysis_chain.invoke({
    "initial_email": EMAIL,
    "email_category": email_category,
    "research_info": research_info,
    "draft_email": draft_email
})

print(email_analysis)

{'draft_analysis': {'effectiveness': 'ineffective', 'feedback': {'original_draft': 'The original draft is brief and to the point, but it lacks specific details about the issue. It would be helpful to include more information about the error message, the course name, and any steps taken so far to troubleshoot the issue.', 'response_draft': "The response draft is unprofessional and unhelpful. It does not acknowledge the customer's issue or provide any solution or guidance. A more empathetic and informative response is needed.", 'improvement_suggestions': ['Original draft: Consider adding more details about the issue, such as the exact error message, the course name, and any steps taken to troubleshoot.', "Response draft: Start with a acknowledgment of the customer's issue and apologize for the inconvenience. Provide a specific solution or guidance on how to resolve the issue, or offer further assistance if needed. Use a professional and empathetic tone."]}}}


In [16]:
rewrite_email_prompt = PromptTemplate(
   template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Final Email Agent for an EdTech platform. Read the email analysis below from the QC Agent \
    and use it to rewrite and improve the draft_email to create a final, professional response that aligns with the values of the platform.

    You must never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Your response should be clear, polite, and informative, reflecting the tone and standards expected of an EdTech company. Please focus on delivering a helpful, thoughtful, and professional reply to the customer.

    Return the final email as JSON with a single key 'final_email' which is a string and no preamble or explanation.

<|eot_id|><|start_header_id|>user<|end_header_id|>
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    DRAFT_EMAIL_FEEDBACK: {email_analysis} \n\n
<|eot_id|><|start_header_id|>"""

)

rewrite_chain = rewrite_email_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'platform_issue'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

final_email = rewrite_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email,
                                "email_analysis":email_analysis})

final_email['final_email']

"Dear [Customer],\n\nI apologize for the inconvenience you're experiencing with our platform. I understand how frustrating it can be when issues arise. Unfortunately, without more information about the error message, the course name, and any steps you've taken to troubleshoot the issue, I'm limited in my ability to assist you. Could you please provide more details about the problem you're facing? This will enable me to better understand the issue and provide a more accurate solution or guidance on how to resolve it. I'm committed to helping you and appreciate your cooperation.\n\nBest regards,\nSarah"

In [17]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=1)

In [18]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [19]:
from typing_extensions import TypedDict
from typing import List, Dict

class GraphState(TypedDict):
    """
    Represents the state of our email handling process on an EdTech platform.

    Attributes:
        initial_email: The user's original email inquiry or request.
        email_category: The category of the email, e.g., 'customer_feedback', 'technical_issue', etc.
        draft_email: The initial draft response generated for the email.
        final_email: The final, improved email response ready for sending.
        research_info: A list of relevant documents, knowledge base, or research gathered to inform the email reply.
        info_needed: A boolean indicating if further research or information gathering is required.
        num_steps: The number of steps involved in processing the email (e.g., initial draft, review, finalization).
        draft_email_feedback: A dictionary containing feedback about the draft email (e.g., tone, clarity).
    """
    initial_email: str  # The original email received from the user.
    email_category: str  # Category like 'customer_feedback', 'course_enquiry', etc.
    draft_email: str  # The initial draft response created by the system.
    final_email: str  # The improved email response after review.
    research_info: List[str]  # List of documents or context used for crafting the response.
    info_needed: bool  # Whether additional research or details are needed.
    num_steps: int  # Number of steps in the email response process (e.g., drafting, feedback, finalization).
    draft_email_feedback: Dict  # Feedback on the draft email (e.g., improvement suggestions, tone analysis).


In [20]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [21]:

def research_info_search(state):

    print("---RESEARCH INFO SEARCHING---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"initial_email": initial_email,
                                            "email_category": email_category })
    keywords = keywords['keywords']
    # print(keywords)
    full_searches = []
    for keyword in keywords[:1]:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(full_searches)
    print(type(full_searches))
    # write_markdown_file(full_searches, "research_info")
    return {"research_info": full_searches, "num_steps":num_steps}

In [22]:
def draft_email_writer(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info":research_info})
    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [23]:
def analyze_draft_email(state):
    print("---DRAFT EMAIL ANALYZER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email_feedback = draft_analysis_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email}
                                               )
    # print(draft_email)
    # print(type(draft_email))

    write_markdown_file(str(draft_email_feedback), "draft_email_feedback")
    return {"draft_email_feedback": draft_email_feedback, "num_steps":num_steps}

In [24]:
def rewrite_email(state):
    print("---ReWRITE EMAIL ---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    draft_email_feedback = state["draft_email_feedback"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    final_email = rewrite_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email,
                                                "email_analysis": draft_email_feedback}
                                               )

    write_markdown_file(str(final_email), "final_email")
    return {"final_email": final_email['final_email'], "num_steps":num_steps}

In [25]:
def no_rewrite(state):
    print("---NO REWRITE EMAIL ---")
    ## Get the state
    draft_email = state["draft_email"]
    num_steps = state['num_steps']
    num_steps += 1

    write_markdown_file(str(draft_email), "final_email")
    return {"final_email": draft_email, "num_steps":num_steps}

In [26]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Draft Email: {state['draft_email']} \n" )
    print(f"Final Email: {state['final_email']} \n" )
    print(f"Research Info: {state['research_info']} \n")
    print(f"Info Needed: {state['info_needed']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

# **EDGES**

In [27]:
def route_to_research(state):
    """
    Route email to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]


    router = research_router.invoke({"initial_email": initial_email,"email_category":email_category })
    print(router)
    # print(type(router))
    print(router['router_decision'])
    if router['router_decision'] == 'research_info':
        print("---ROUTE EMAIL TO RESEARCH INFO---")
        return "research_info"
    elif router['router_decision'] == 'draft_email':
        print("---ROUTE EMAIL TO DRAFT EMAIL---")
        return "draft_email"

In [28]:
def route_to_rewrite(state):

    print("---ROUTE TO REWRITE---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]

    # draft_email = "Yo we can't help you, best regards Sarah"

    router = rewrite_router.invoke({"initial_email": initial_email,
                                     "email_category":email_category,
                                     "draft_email":draft_email}
                                   )
    print(router)
    print(router['router_decision'])
    if router['router_decision'] == 'rewrite':
        print("---ROUTE TO ANALYSIS - REWRITE---")
        return "rewrite"
    elif router['router_decision'] == 'no_rewrite':
        print("---ROUTE EMAIL TO FINAL EMAIL---")
        return "no_rewrite"

In [29]:

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_email", categorize_email) # categorize email
workflow.add_node("research_info_search", research_info_search) # web search
workflow.add_node("state_printer", state_printer)
workflow.add_node("draft_email_writer", draft_email_writer)
workflow.add_node("analyze_draft_email", analyze_draft_email)
workflow.add_node("rewrite_email", rewrite_email)
workflow.add_node("no_rewrite", no_rewrite)



In [30]:
workflow.set_entry_point("categorize_email")

workflow.add_conditional_edges(
    "categorize_email",
    route_to_research,
    {
        "research_info": "research_info_search",
        "draft_email": "draft_email_writer",
    },
)
workflow.add_edge("research_info_search", "draft_email_writer")


workflow.add_conditional_edges(
    "draft_email_writer",
    route_to_rewrite,
    {
        "rewrite": "analyze_draft_email",
        "no_rewrite": "no_rewrite",
    },
)
workflow.add_edge("analyze_draft_email", "rewrite_email")
workflow.add_edge("no_rewrite", "state_printer")
workflow.add_edge("rewrite_email", "state_printer")
workflow.add_edge("state_printer", END)

In [31]:
# Compile
app = workflow.compile()

In [32]:
EMAIL = """HI there, \n
I am emailing to say that the recent course on Data Science had a lot of challenges in terms of accessibility. \n
I had trouble accessing some of the course materials, which made it difficult to complete the assignments. Could you provide more information about how the course will be made more accessible in the future?

Also, can you clarify what the typical pace is for this course, as I would like to ensure I can keep up?

I hope this feedback helps improve the course for future students.

Thanks,
George
"""


In [33]:
# run the agent
inputs = {"initial_email": EMAIL,"research_info": None, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
feedback
---ROUTE TO RESEARCH---
{'router_decision': 'research_info'}
research_info
---ROUTE EMAIL TO RESEARCH INFO---
Finished running: categorize_email:
---RESEARCH INFO SEARCHING---
course accessibility
[Document(metadata={}, page_content="Online Web Accessibility courses offer a convenient and flexible way to enhance your knowledge or learn new Web Accessibility skills. Choose from a wide range of Web Accessibility courses offered by top universities and industry leaders tailored to various skill levels.\nExplore edX's accessibility courses to learn how to create inclusive and accessible digital content for all users.\nAn evergreen accessibility course and reference to level up your web development. ... Introduction to accessibility measurement, which ensures everyone, including people with disabilities, can still interact with your website in a meaningful and equal way.\nDigital Accessibility Courses Overview The following information about digital

OutputParserException: Invalid json output: Subject: Re: Feedback on Data Science Course Accessibility

Dear George,

Thank you so much for taking the time to share your valuable feedback about the recent Data Science course. We appreciate your honesty and are sorry to hear that you experienced accessibility challenges with the course materials. Your input is invaluable in helping us improve the overall learning experience for our students.

We are committed to making our courses more accessible and inclusive for everyone. We are actively working on enhancing the accessibility features of our platform to ensure that all students can easily access and engage with the course content. We'll make sure to provide clear guidelines and alternative formats for students who require them.

Regarding your question about the typical pace of the course, our Data Science course is designed to be self-paced, allowing you to learn at your own speed. However, we recommend allocating a specific time eac